In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import gurobipy as gp
print(gp.gurobi.version())

(12, 0, 0)


In [6]:
pip install --upgrade numpy pandas


Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## load data

In [ ]:
# # Load data files
# child_care = pd.read_csv('/content/drive/My Drive/Optimization Projects/project3/child_care_regulated.csv')
# population = pd.read_csv('/content/drive/My Drive/Optimization Projects/project3/population.csv')
# income = pd.read_csv('/content/drive/My Drive/Optimization Projects/project3/avg_individual_income.csv')
# employment = pd.read_csv('/content/drive/My Drive/Optimization Projects/project3/employment_rate.csv')
# potential_locations = pd.read_csv('/content/drive/My Drive/Optimization Projects/project3/potential_locations.csv')

In [3]:
# Load data files
child_care = pd.read_csv('child_care_regulated.csv')
population = pd.read_csv('population.csv')
income = pd.read_csv('avg_individual_income.csv')
employment = pd.read_csv('employment_rate.csv')
potential_locations = pd.read_csv('potential_locations.csv')

In [4]:
population.head()

,zipcode,Total,-5,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,6390,53,0,1,5,0,6,0,9,18,0,12,2,0,0,0,0,0,0,0
1,10001,27004,744,784,942,1035,2296,3806,3588,2524,1702,1903,1704,1225,1323,933,815,616,488,576
2,10002,76518,2142,3046,3198,2652,4528,6988,6278,5157,4962,4822,4410,6106,4548,4815,4748,2531,2793,2794
3,10003,53877,1440,1034,953,7013,6344,7100,6427,3221,2907,1988,2698,2350,2274,2793,1854,1646,779,1056
4,10004,4579,433,182,161,108,109,601,724,490,241,313,549,279,199,173,2,15,0,0


## Pre-process

In [5]:
# rename
population.rename(columns={'zipcode': 'zip_code'}, inplace=True)
income.rename(columns={'ZIP code': 'zip_code'}, inplace=True)
employment.rename(columns={'zipcode': 'zip_code'}, inplace=True)
potential_locations.rename(columns={'zipcode': 'zip_code'}, inplace=True)

# Merge relevant datasets on zip_code
merged_data = population.merge(child_care, on='zip_code', how='left')
merged_data = merged_data.merge(income, on='zip_code', how='left')
merged_data = merged_data.merge(employment, on='zip_code', how='left')
merged_data['total_capacity'].fillna(0, inplace=True)

/var/folders/x4/bz8s0r0d2q55wsw46_t802l00000gn/T/ipykernel_10756/2889968728.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['total_capacity'].fillna(0, inplace=True)


In [6]:
merged_data.head()

,zip_code,Total,-5,5-9,10-14,15-19,20-24,25-29,30-34,35-39,...,infant_capacity,toddler_capacity,preschool_capacity,school_age_capacity,children_capacity,total_capacity,latitude,longitude,average income,employment rate
0,6390,53,0,1,5,0,6,0,9,18,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,10001,27004,744,784,942,1035,2296,3806,3588,2524,...,0.0,0.0,0.0,84.0,0.0,84.0,40.748836,-73.999810,102878.033603,0.595097
2,10001,27004,744,784,942,1035,2296,3806,3588,2524,...,0.0,0.0,0.0,4.0,12.0,16.0,40.748911,-74.001546,102878.033603,0.595097
3,10001,27004,744,784,942,1035,2296,3806,3588,2524,...,0.0,0.0,0.0,17.0,0.0,17.0,40.752093,-74.002588,102878.033603,0.595097
4,10001,27004,744,784,942,1035,2296,3806,3588,2524,...,0.0,0.0,0.0,2.0,6.0,8.0,40.748296,-74.001263,102878.033603,0.595097


In [7]:
# Define high and normal demand areas
merged_data['high_demand'] = (merged_data['employment rate'] >= 60) | (merged_data['average income'] <= 60000)

# Compute total child population and current child care coverage
merged_data['total_children'] = merged_data['-5'] + merged_data['5-9']
merged_data['coverage_ratio'] = merged_data['total_capacity'] / merged_data['total_children']

# Identify child care deserts
merged_data['child_care_desert'] = ((merged_data['high_demand'] & (merged_data['coverage_ratio'] <= 0.5)) |
                                    (~merged_data['high_demand'] & (merged_data['coverage_ratio'] <= 0.33)))

## Optimization Model

### pulp

In [8]:
!pip install pulp


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from pulp import *

#### pulp+gurobi

In [12]:
# 1. Load Data
zip_codes = merged_data['zip_code'].unique()
facility_types = [1, 2, 3]  # small 1 medium 2 large 3

# Facility cost and capacity
facility_cost = {1: 65000, 2: 95000, 3: 115000}
facility_capacity = {1: 100, 2: 200, 3: 400}

# Expansion cost
expansion_base_cost = 20000
expansion_per_slot_cost = 200
under5_extra_cost = 100  # Extra cost for 0-5

# 2. Build PuLP Linear Optimization Model
model = LpProblem("Child_Care_Budget_Optimization", LpMinimize)

# 3. Define decision variables
# Number of new facilities to build
y = LpVariable.dicts("NewFacility", ((z, i) for z in zip_codes for i in facility_types),
                      cat='Integer', lowBound=0)

# Facility expansion ratio (maximum 20%)
x = LpVariable.dicts("ExpansionRatio", zip_codes, lowBound=0, upBound=0.2)

# Additional 0–5 year old child care capacity
mu = LpVariable.dicts("Under5Expansion", zip_codes, lowBound=0)

# 4. Objective function: Minimize total cost
model += (
    lpSum([y[z, i] * facility_cost[i] for z in zip_codes for i in facility_types]) +
    lpSum([
        (expansion_base_cost + expansion_per_slot_cost * merged_data.loc[merged_data['zip_code'] == z, 'total_capacity'].values[0]) * x[z]
        + under5_extra_cost * mu[z]
        for z in zip_codes
    ])
)

# 5. Constraints
for idx, row in merged_data.iterrows():
    z = row['zip_code']
    total_children = row['-5'] + row['5-9']  # Calculate total number of children aged 0–12
    under5_population = row['-5']
    current_capacity = row['total_capacity']
    is_high_demand = row['high_demand']

    # Calculate total capacity (existing + expanded + new)）
    new_capacity = (
        lpSum([facility_capacity[i] * y[z, i] for i in facility_types]) +
        current_capacity + current_capacity * x[z]
    )

    # Constraint 1: Ensure all areas meet basic child care needs
    if is_high_demand:
        model += new_capacity >= 0.5 * total_children, f"HighDemand_Coverage_{z}_{idx}"
    else:
        model += new_capacity >= (1/3) * total_children, f"NormalDemand_Coverage_{z}_{idx}"

    # Constraint 2: At least 2/3 of under-5 children must be covered
    new_under5_capacity = (
        lpSum([0.5 * facility_capacity[i] * y[z, i] for i in facility_types]) +
        mu[z]
    )
    model += new_under5_capacity >= (2/3) * under5_population, f"Under5_Coverage_{z}_{idx}"

    # Constraint 3: Under-5 expansion must not exceed total expansion amount
    model += mu[z] <= current_capacity * x[z], f"Under5_Limit_{z}_{idx}"

# 6. Solve the model
solver = GUROBI_CMD(path='/usr/local/bin/gurobi_cl', msg=1)
model.solve(solver)

# 7. Output
# Print optimal cost
print(f"Total Minimum Cost: ${value(model.objective):,.2f}")

print("Fianl Result")
for z in zip_codes:
    small = y[z, 1].varValue if y[z, 1].varValue else 0
    medium = y[z, 2].varValue if y[z, 2].varValue else 0
    large = y[z, 3].varValue if y[z, 3].varValue else 0
    expand = x[z].varValue * 100 if x[z].varValue else 0

    print(f"{z} → small: {small:.1f}, medium: {medium:.1f}, large: {large:.1f}, expand: {expand:.2f}%")


Set parameter Username
Set parameter LicenseID to value 2642225
Set parameter LogFile to value "gurobi.log"
Using license file /Users/jiallu/gurobi.lic
Academic license - for non-commercial use only - expires 2026-03-25

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)
Copyright (c) 2025, Gurobi Optimization, LLC

Read LP format model from file /var/folders/x4/bz8s0r0d2q55wsw46_t802l00000gn/T/91b0f98d16ad4be99f640c81622f087b-pulp.lp
Reading time = 0.06 seconds
OBJ: 45261 rows, 8230 columns, 149686 nonzeros

Using Gurobi shared library /Library/gurobi1201/macos_universal2/lib/libgurobi120.dylib

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45261 rows, 8230 columns and 149686 nonzeros
Model fingerprint: 0xea06eb9c
Variable types: 3292 continuous, 4938 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [

#### pulp+CBC_CMD

In [ ]:
# 1. Load Data
zip_codes = merged_data['zip_code'].unique()
facility_types = [1, 2, 3]  # 小型（1），中型（2），大型（3）

# Facility cost and capacity
facility_cost = {1: 65000, 2: 95000, 3: 115000}
facility_capacity = {1: 100, 2: 200, 3: 400}

# Expansion cost
expansion_base_cost = 20000
expansion_per_slot_cost = 200
under5_extra_cost = 100  # Extra cost for 0-5

# 2. Build PuLP Linear Optimization Model
model = LpProblem("Child_Care_Budget_Optimization", LpMinimize)

# 3. Define decision variables
# Number of new facilities to build
y = LpVariable.dicts("NewFacility", ((z, i) for z in zip_codes for i in facility_types),
                      cat='Integer', lowBound=0)

# Facility expansion ratio (maximum 20%)
x = LpVariable.dicts("ExpansionRatio", zip_codes, lowBound=0, upBound=0.2)

# Additional 0–5 year old child care capacity
mu = LpVariable.dicts("Under5Expansion", zip_codes, lowBound=0)

# 4. Objective function: Minimize total cost
model += (
    lpSum([y[z, i] * facility_cost[i] for z in zip_codes for i in facility_types]) +
    lpSum([
        (expansion_base_cost + expansion_per_slot_cost * merged_data.loc[merged_data['zip_code'] == z, 'total_capacity'].values[0]) * x[z]
        + under5_extra_cost * mu[z]
        for z in zip_codes
    ])
)

# 5. Constraints
for idx, row in merged_data.iterrows():
    z = row['zip_code']
    total_children = row['-5'] + row['5-9']  # Calculate total number of children aged 0–12
    under5_population = row['-5']
    current_capacity = row['total_capacity']
    is_high_demand = row['high_demand']

    # Calculate total capacity (existing + expanded + new)）
    new_capacity = (
        lpSum([facility_capacity[i] * y[z, i] for i in facility_types]) +
        current_capacity + current_capacity * x[z]
    )

    # Constraint 1: Ensure all areas meet basic child care needs
    if is_high_demand:
        model += new_capacity >= 0.5 * total_children, f"HighDemand_Coverage_{z}_{idx}"
    else:
        model += new_capacity >= (1/3) * total_children, f"NormalDemand_Coverage_{z}_{idx}"

    # Constraint 2: At least 2/3 of under-5 children must be covered
    new_under5_capacity = (
        lpSum([0.5 * facility_capacity[i] * y[z, i] for i in facility_types]) +
        mu[z]
    )
    model += new_under5_capacity >= (2/3) * under5_population, f"Under5_Coverage_{z}_{idx}"

    # Constraint 3: Under-5 expansion must not exceed total expansion amount
    model += mu[z] <= current_capacity * x[z], f"Under5_Limit_{z}_{idx}"

# 6. Solve the model
solver = PULP_CBC_CMD(msg=1)
model.solve(solver)

# 7. Output
# Print optimal cost
print(f"Total Minimum Cost: ${value(model.objective):,.2f}")

print("Fianl Result")
for z in zip_codes:
    small = y[z, 1].varValue if y[z, 1].varValue else 0
    medium = y[z, 2].varValue if y[z, 2].varValue else 0
    large = y[z, 3].varValue if y[z, 3].varValue else 0
    expand = x[z].varValue * 100 if x[z].varValue else 0

    print(f"{z} → small: {small:.1f}, medium: {medium:.1f}, large: {large:.1f}, expand: {expand:.2f}%")



Total Minimum Cost: $472,632,298.62
Fianl Result
6390 → small: 1.0, medium: 0.0, large: 0.0, expand: 0.00%
10001 → small: 0.0, medium: 1.0, large: 2.0, expand: 0.00%
10002 → small: 1.0, medium: 0.0, large: 7.0, expand: 0.00%
10003 → small: 0.0, medium: 0.0, large: 5.0, expand: 0.00%
10004 → small: 0.0, medium: 1.0, large: 1.0, expand: 0.00%
10005 → small: 0.0, medium: 0.0, large: 2.0, expand: 0.00%
10006 → small: 0.0, medium: 1.0, large: 0.0, expand: 0.00%
10007 → small: 0.0, medium: 0.0, large: 2.0, expand: 19.61%
10009 → small: 0.0, medium: 1.0, large: 6.0, expand: 0.00%
10010 → small: 0.0, medium: 0.0, large: 5.0, expand: 0.00%
10011 → small: 0.0, medium: 0.0, large: 4.0, expand: 3.41%
10012 → small: 1.0, medium: 0.0, large: 2.0, expand: 0.00%
10013 → small: 1.0, medium: 0.0, large: 4.0, expand: 0.00%
10014 → small: 1.0, medium: 0.0, large: 2.0, expand: 15.71%
10016 → small: 0.0, medium: 0.0, large: 6.0, expand: 0.00%
10017 → small: 0.0, medium: 0.0, large: 1.0, expand: 0.00%
10018 

### gurobi

In [15]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# --- 1. Load preprocessed merged_data ---
# merged_data = pd.read_csv('your_cleaned_file.csv')
zip_codes = merged_data['zip_code'].unique()
facility_types = [1, 2, 3]  # 1: small, 2: medium, 3: large

# --- 2. Model initialization ---
model = gp.Model("ChildCareOptimization")

# --- 3. Parameters ---
facility_cost = {1: 65000, 2: 95000, 3: 115000}
facility_capacity = {1: 100, 2: 200, 3: 400}
expansion_base_cost = 20000
expansion_per_slot_cost = 200
under5_extra_cost = 100

# --- 4. Decision Variables ---
# New facilities: integer
y = model.addVars(zip_codes, facility_types, vtype=GRB.INTEGER, name="NewFacility")
# Expansion ratio: continuous [0, 0.2]
x = model.addVars(zip_codes, lb=0, ub=0.2, vtype=GRB.CONTINUOUS, name="ExpansionRatio")
# New under-5 slots: continuous
mu = model.addVars(zip_codes, lb=0, vtype=GRB.CONTINUOUS, name="Under5Expansion")

# --- 5. Objective: minimize total cost ---
model.setObjective(
    gp.quicksum(y[z, i] * facility_cost[i] for z in zip_codes for i in facility_types) +
    gp.quicksum((expansion_base_cost + expansion_per_slot_cost * merged_data.loc[merged_data['zip_code'] == z, 'total_capacity'].values[0]) * x[z] +
                under5_extra_cost * mu[z] for z in zip_codes),
    GRB.MINIMIZE
)

# --- 6. Constraints ---
for idx, row in merged_data.iterrows():
    z = row['zip_code']
    total_children = row['-5'] + row['5-9']
    under5_population = row['-5']
    current_capacity = row['total_capacity']
    is_high_demand = row['high_demand']

    new_capacity = (
        gp.quicksum(facility_capacity[i] * y[z, i] for i in facility_types) +
        current_capacity + current_capacity * x[z]
    )

    # Child care desert elimination
    if is_high_demand:
        model.addConstr(new_capacity >= 0.5 * total_children, f"HighDemand_{z}")
    else:
        model.addConstr(new_capacity >= (1 / 3) * total_children, f"NormalDemand_{z}")

    # Under-5 coverage
    under5_capacity = (
        gp.quicksum(0.5 * facility_capacity[i] * y[z, i] for i in facility_types) +
        mu[z]
    )
    model.addConstr(under5_capacity >= (2 / 3) * under5_population, f"Under5_{z}")

    # Under-5 new slots ≤ total expansion
    model.addConstr(mu[z] <= current_capacity * x[z], f"Under5Limit_{z}")

# --- 7. Optimize ---
model.optimize()

# --- 8. Print Results ---
print(f"\n Total Minimum Cost: ${model.ObjVal:,.2f}")

print("\n Final Results")
for z in zip_codes:
    small = y[z, 1].X if y[z, 1].X else 0
    medium = y[z, 2].X if y[z, 2].X else 0
    large = y[z, 3].X if y[z, 3].X else 0
    expand = x[z].X * 100 if x[z].X else 0
    print(f"{z} → small: {small:.1f}, medium: {medium:.1f}, large: {large:.1f}, expand: {expand:.2f}%")



Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45261 rows, 8230 columns and 149686 nonzeros
Model fingerprint: 0xb8957eba
Variable types: 3292 continuous, 4938 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [2e-01, 2e-01]
  RHS range        [3e-01, 1e+04]
Found heuristic solution: objective 5.180750e+08
Presolve removed 45258 rows and 8225 columns
Presolve time: 0.09s
Presolved: 3 rows, 5 columns, 10 nonzeros
Found heuristic solution: objective 4.758447e+08
Variable types: 0 continuous, 5 integer (1 binary)

Explored 0 nodes (0 simplex iterations) in 0.11 seconds (0.09 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 4.75845e+08 5.18075e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.758446746299e+08, b

### Way2

In [13]:
# Define optimization model
model = Model("ChildCareExpansion")

# Decision variables
expand_facility = {j: model.addVar(lb=0, ub=0.2, vtype=GRB.CONTINUOUS, name=f"expand_{j}") for j in merged_data.index}
build_facility = {i: model.addVar(vtype=GRB.INTEGER, name=f"build_{i}") for i in potential_locations.index}

# Add additional binary variables to indicate the stage of expansion
z1 = {j: model.addVar(vtype=GRB.BINARY, name=f"z1_{j}") for j in merged_data.index}  # 0-10%
z2 = {j: model.addVar(vtype=GRB.BINARY, name=f"z2_{j}") for j in merged_data.index}  # 10-15%
z3 = {j: model.addVar(vtype=GRB.BINARY, name=f"z3_{j}") for j in merged_data.index}  # 15-20%

# Constraints:
for j in merged_data.index:
    model.addConstr(z1[j] + z2[j] + z3[j] == 1)  # Ensure each facility belongs to only one expansion phase

# Define Cost
expansion_cost = sum(
    (20000 + 200 * merged_data.loc[j, 'total_capacity']) * expand_facility[j] * z1[j] +
    (20000 + 400 * merged_data.loc[j, 'total_capacity']) * expand_facility[j] * z2[j] +
    (20000 + 1000 * merged_data.loc[j, 'total_capacity']) * expand_facility[j] * z3[j]
    for j in merged_data.index
)

new_facility_cost = sum(95000 * build_facility[i] for i in potential_locations.index)

# Objective function: Minimize total cost
model.setObjective(expansion_cost + new_facility_cost, GRB.MINIMIZE)

# Other constraints (e.g. coverage requirements, budget, etc.)
for idx, row in merged_data.iterrows():
    if row['child_care_desert']:
        required_capacity = 0.5 * row['total_children'] if row['high_demand'] else 0.33 * row['total_children']
        model.addConstr(
            merged_data.loc[idx, 'total_capacity'] + expand_facility[idx] * merged_data.loc[idx, 'total_capacity'] +
            sum(build_facility[i] * 200 for i in potential_locations[potential_locations['zip_code'] == row['zip_code']].index) >= required_capacity
        )

# Budget Constraints
model.addConstr(expansion_cost + new_facility_cost <= 100000000)

Set parameter Username
Set parameter LicenseID to value 2642225
Academic license - for non-commercial use only - expires 2026-03-25


<gurobi.QConstr Not Yet Added>

In [14]:
# Solve
model.optimize()

# Extract results
results = {
    'zip_code': [], 'expand_ratio': [], 'new_facilities': []
}
for idx in merged_data.index:
    results['zip_code'].append(merged_data.loc[idx, 'zip_code'])
    results['expand_ratio'].append(expand_facility[idx].X)
for i in potential_locations.index:
    results['new_facilities'].append(build_facility[i].X)

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
# Display results
tools.display_dataframe_to_user(name="Optimization Results", dataframe=results_df)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29874 rows, 275748 columns and 1538311 nonzeros
Model fingerprint: 0x6ad0003c
Model has 45261 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 15087 continuous, 260661 integer (45261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  QMatrix range    [2e+04, 9e+05]
  QLMatrix range   [1e+05, 1e+05]
  Objective range  [1e+05, 1e+05]
  QObjective range [4e+04, 2e+06]
  Bounds range     [2e-01, 1e+00]
  RHS range        [1e-02, 1e+04]
  QRHS range       [1e+08, 1e+08]
Presolve removed 11876 rows and 74006 columns
Presolve time: 0.53s
Presolved: 63260 rows, 247003 columns, 564697 nonzeros
Variable types: 7976 continuous, 239027 integer (45261 binary)

Explored 1 nodes (0 simplex iterations) in 0.88 seconds (1.71 work units)
Thread count was 8 (of 

AttributeError: Unable to retrieve attribute 'X'

In [ ]:
print(f"Number of Variables: {model.NumVars}")
print(f"Number of Constraints: {model.NumConstrs}")


Number of Variables: 275748
Number of Constraints: 29875
